# Project2 - Two-Node Network Analysis

**GROUP: Forhad Akbar, Adam Douglas, and Soumya Ghosh**

## Project Overview
1. Identify a large 2-node network dataset—you can start with a dataset in a repository.  Your data should meet the criteria that it consists of ties between and not within two (or more) distinct groups.
2. Reduce the size of the network using a method such as the island method described in chapter 4 of social network analysis.
3. What can you infer about each of the distinct groups?

## Datset Background

In the 1990s Rick Rosenfeld and Norm White used police records to collect data on crime in St. Louis. They began with five homicides and recorded the names of all the individuals who had been involved as victims, suspects or witnesses. They then explored the files and recorded all the other crimes in which those same individuals appeared. This snowball process was continued until they had data on 557 crime events. Those events involved 870 participants of which: 569 appeared as victims 682 appeared as suspects 195 appeared as witnesses, and 41 were dual (they were recorded both as victims and suspects in the same crime. Their data appear, then, as an 870 by 557, individual by crime event matrix. Victims are coded as 1, suspects as 2, witnesses as 3 and duals as 4. In addition Rosenfeld and White recorded the sex of each individual.

### Data Sources:

https://github.com/nderzsy/Network-Analysis-in-Python---Tutorial-JupyterCon18-ODSCEast18/tree/master/datafiles/social/crime

http://moreno.ss.uci.edu/data.html#crime

## Data Import

We begin by importing our data from the source site (github). The data is provided in several files, so there are a few steps that need to be taken to put the data together into a single bipartite graph.

In [20]:
# Import all necessary packages

import pandas as pd
import numpy as np
import math
import networkx as nx
import matplotlib.pyplot as plt
from functools import reduce
%matplotlib inline
import networkx.algorithms.bipartite as bipartite
from pyvis import network as net
import matplotlib.pyplot as plt

In [21]:
# Read in persons and their associated sex

person = pd.read_csv('Dataset/ent.moreno_crime_crime.person.name', sep='\t', header = None, names = ['Name'])
person['Sex'] = pd.read_csv('Dataset/ent.moreno_crime_crime.person.sex', header = None)

person.loc[person.Sex == 0, ['Sex']] = 'F'
person.loc[person.Sex == 1, ['Sex']] = 'M'

In [22]:
# Create the crime dataframe that associates a person with a crime in some manner

crime = pd.read_csv("Dataset/out.moreno_crime_crime", delim_whitespace = True, skiprows = [0,1], names = ['Person', 'Crime'])

In [23]:
# Add the roles of each person (e.g. Person 1 is the suspect in Crime 1)

crime['Role'] = pd.read_csv("Dataset/rel.moreno_crime_crime.person.role", header = None)

In [24]:
# Add the name and sex to the main crime dataframe

crime["Name"] = ""
crime["Sex"] = ""

for i in range(0, len(person)):
    crime.loc[crime.Person == i+1, ['Sex']] = person.iloc[i]["Sex"]
    crime.loc[crime.Person == i+1, ['Name']] = person.iloc[i]["Name"]

# Change the crime event to a string and append "C"
crime = crime.astype({"Crime": str})
crime["Crime"] = "C" + crime["Crime"]

# Remove witness entries
crime.drop(crime[crime['Role'] == "Witness"].index, inplace = True) 

#Final dataset
crime.head()

,Person,Crime,Role,Name,Sex
0,1,C1,Suspect,AbelDennis,M
1,1,C2,Victim,AbelDennis,M
2,1,C3,Victim,AbelDennis,M
3,1,C4,Suspect,AbelDennis,M
4,2,C5,Victim,AbramsChad,M


Now that we have the data in a combined dataframe, we can begin to put it together into a graph format.

## Creating the Graph
We are going to create the graph with following features -
 1. A person (victim or suspect) will be reperesented as a node with *'bipartite' = 0*
   - For male, the color code is *'dodgerblue'*
   - For female, the color code is *'teal'*
   
 2. Each crime occurence will be represnted as a node with *'bipartite' = 1* and 'red' as the color code
 3. A *suspect* will be repesneted by an inbound edge towards the crime event
 4. A *victim* will be represented by an outbound edge from the crime event

In [25]:
colors = {"M":"dodgerblue","F":"teal"}

G = nx.DiGraph()

for n, r, c, s in zip(crime['Name'],crime['Role'],crime['Crime'],crime['Sex']):
    if n not in G.nodes() and r != 'Witness':
        G.add_node(n, Sex = s, bipartite = 0, color = colors[s])
    if c not in G.nodes():
        G.add_node(c, bipartite = 1, color = 'red')
    if r == 'Suspect':
        G.add_edge(n,c)
    elif r == 'Victim':
        G.add_edge(c,n)

print(nx.info(G))

Name: 
Type: DiGraph
Number of nodes: 1259
Number of edges: 1240
Average in degree:   0.9849
Average out degree:   0.9849


In [26]:
nx.is_bipartite(G)

True

In [27]:
list(G.nodes.data())[1:10]

[('C1', {'bipartite': 1, 'color': 'red'}),
 ('C2', {'bipartite': 1, 'color': 'red'}),
 ('C3', {'bipartite': 1, 'color': 'red'}),
 ('C4', {'bipartite': 1, 'color': 'red'}),
 ('AbramsChad', {'Sex': 'M', 'bipartite': 0, 'color': 'dodgerblue'}),
 ('C5', {'bipartite': 1, 'color': 'red'}),
 ('C6', {'bipartite': 1, 'color': 'red'}),
 ('C7', {'bipartite': 1, 'color': 'red'}),
 ('C8', {'bipartite': 1, 'color': 'red'})]

In [28]:
n = net.Network(height = "1000px", width = "100%", notebook = True, 
                bgcolor = "#ffffff", font_color = "black",
                heading = 'Crimes and People', directed = True)
nx_graph = nx.Graph(G)
n.from_nx(nx_graph, default_node_size = 50, default_edge_weight = 1)
n.show_buttons(filter_=['physics'])
n.show("crimenet.html")

The first thing that jumps out is the number of nodes where there are no connections. Because we specifically excluded the witnesses, this should not be.

Let's look and see why that is: 

In [29]:
list(nx.isolates(G))

['C34',
 'AndrewsSally',
 'C54',
 'BeckerMax',
 'C146',
 'BoyleAlice',
 'CanfieldAristides',
 'C192',
 'C198',
 'C228',
 'ConrackCarol',
 'CoreyAlonzo',
 'C241',
 'C271',
 'EvansJay',
 'C311',
 'FindlayGary',
 'C318',
 'ForesterCarol',
 'GleesonMatt',
 'C341',
 'GuntherMatt',
 'KetterPercy',
 'KirklandNiles',
 'KirklandRudy',
 'C439',
 'C474',
 'NoblesCary',
 'ReddickJohn',
 'RosenHenry',
 'SprintMelody',
 'StanleyMaurice',
 'StithCarlton',
 'TillieNigel',
 'C324',
 'TraskBenjie']

In [30]:
unconn = [n for n in nx.isolates(G)]

isol = [(n, c, r) for n, c, r in zip(crime["Name"],crime["Crime"],crime["Role"]) if c in unconn or n in unconn]

sorted(isol, key = lambda x: x[1], reverse = True)

[('BeckerMax', 'C86', 'Victim Suspect'),
 ('TraskBenjie', 'C86', 'Victim Suspect'),
 ('ReddickJohn', 'C82', 'Victim Suspect'),
 ('AndrewsSally', 'C54', 'Victim Suspect'),
 ('LeeCrystal', 'C54', 'Victim Suspect'),
 ('MitchellGrant', 'C474', 'Victim Suspect'),
 ('WinfreyAlice', 'C474', 'Victim Suspect'),
 ('KirklandRudy', 'C439', 'Victim Suspect'),
 ('VestMiko', 'C439', 'Victim Suspect'),
 ('KetterPercy', 'C436', 'Victim Suspect'),
 ('StanleyMaurice', 'C390', 'Victim Suspect'),
 ('GleesonMatt', 'C341', 'Victim Suspect'),
 ('GliddenDiana', 'C341', 'Victim Suspect'),
 ('AlexanderNolan', 'C34', 'Victim Suspect'),
 ('BendixJerryLee', 'C34', 'Victim Suspect'),
 ('TillieNigel', 'C324', 'Victim Suspect'),
 ('WolffRodney', 'C324', 'Victim Suspect'),
 ('FindlayGary', 'C318', 'Victim Suspect'),
 ('JeffersonMillie', 'C318', 'Victim Suspect'),
 ('NoblesCary', 'C318', 'Victim Suspect'),
 ('EvansJay', 'C311', 'Victim Suspect'),
 ('TerryThomas', 'C311', 'Victim Suspect'),
 ('DerbyTammy', 'C271', 'Victi

It appears that these unconnected nodes are where the person is both a victim and a suspect. Why would such a thing occur?

Well, one situation might be when the crime is a fight or sorts where both parties are responsible. In those cases, we would see more than one person in that role (e.g. C86). However we see a few that only have a single person. So, unless that person is fighting themselves (see *Fight Club*), that makes no sense.

Let's look at an example:

In [31]:
crime[crime["Crime"]=="C82"]

,Person,Crime,Role,Name,Sex
570,319,C82,Suspect,GreenByron,M
1028,567,C82,Suspect,OneilLinda,F
1120,632,C82,Victim Suspect,ReddickJohn,M
1365,772,C82,Victim,TylerOwen,M


Now we see a bit more clearly. In the above example the person "ReddickJohn" is not the only participant in the crime. Perhaps the authorities suspect that this person was "in on" the crime despite their attempts to present themselves as another victim?

We should split these entries into 2, one as a suspect and one as a victim.

In [32]:
# Split the role column

crime = crime.drop('Role', axis=1).join(crime['Role'].str.split(' ', expand=True).stack().reset_index(level=1, drop=True).rename('Role'))

# Check our example from above
crime[crime["Crime"]=="C82"]

,Person,Crime,Name,Sex,Role
570,319,C82,GreenByron,M,Suspect
1028,567,C82,OneilLinda,F,Suspect
1120,632,C82,ReddickJohn,M,Victim
1120,632,C82,ReddickJohn,M,Suspect
1365,772,C82,TylerOwen,M,Victim


Now we see that "ReddickJohn" is listed twice, once as a Victim and once as a Suspect. We will need to regenerate the graph to show this update:

In [33]:
G = nx.DiGraph()

for n, r, c, s in zip(crime['Name'],crime['Role'],crime['Crime'],crime['Sex']):
    if n not in G.nodes() and r != 'Witness':
        G.add_node(n, Sex = s, bipartite = 0, color = colors[s])
    if c not in G.nodes():
        G.add_node(c, bipartite = 1, color = 'red')
    if r == 'Suspect':
        G.add_edge(n,c)
    elif r == 'Victim':
        G.add_edge(c,n)

print(nx.info(G))

Name: 
Type: DiGraph
Number of nodes: 1259
Number of edges: 1322
Average in degree:   1.0500
Average out degree:   1.0500


In [34]:
n2 = net.Network(height = "1000px", width = "100%", notebook = True, 
                bgcolor = "#ffffff", font_color = "black",
                heading = 'Crimes and People V2', directed = True)
nx_graph = nx.Graph(G)
n2.from_nx(nx_graph, default_node_size = 50, default_edge_weight = 1)
n2.show_buttons(filter_=['physics'])
n2.show("crimenet2.html")

This looks MUCH better. Let's check for isolates again:

In [35]:
unconn = [n for n in nx.isolates(G)]

isol = [(n, c, r) for n, c, r in zip(crime["Name"],crime["Crime"],crime["Role"]) if c in unconn or n in unconn]

sorted(isol, key = lambda x: x[1], reverse = True)

[]

As we hoped, there are none.

## Sub-Graph Analysis - People & Crime
Next, we wanted to create sub-graphs for both people (victim/suspect) and crimes and analyze them separately.

In [36]:
top_nodes = {n for n, d in G.nodes(data=True) if d["bipartite"] == 0}
bottom_nodes = set(G) - top_nodes

P = bipartite.weighted_projected_graph(G, top_nodes)
C = bipartite.weighted_projected_graph(G, bottom_nodes)

Let's visualize them both -

In [37]:
n_p = net.Network(height = "800px", width = "100%", notebook = True,
               heading = 'People', directed = True)

for n, v in P.nodes(data = True):
    n_p.add_node(n, color = colors[v["Sex"]])
        
for u,v,d in P.edges(data=True):
    n_p.add_edge(u,v,value=d['weight'])
    
n_p.show("graph.html")

From the graph above, it looks like there are quite a few people who are not connected with others which could be a result of no suspect being identified. But there are quite few cluster of nodes where multiple individuals are closely associated.

In [38]:
n_c = net.Network(height = "800px", width = "100%", notebook = True,
               heading = 'Crimes', directed = True)

for n, v in C.nodes(data = True):
    n_c.add_node(n, color = 'red')
        
for u,v,d in C.edges(data=True):
    n_c.add_edge(u,v,value=d['weight'])
    
n_c.show("graph.html")

### People (Suspect/Victim)
Next, we will look at the people more specifically.

First we will look at the out degree. That is, the count of times someone is listed as a suspect in a crime:

In [45]:
suspect_df = pd.DataFrame(sorted(P.out_degree(), key = lambda x: x[1], reverse=True), columns=['Person','Out Degree'])
suspect_df.head(10)

,Person,Out Degree
0,WillisJenny,14
1,CarverJustin,10
2,HemphillBud,8
3,CarverJason,7
4,BallMelvin,7
5,MunsonKaty,6
6,AbramsChad,6
7,TuckerPaul,6
8,DicksonCarter,6
9,BurnleyAaron,6


Next we will look at the opposite, the in degree, or how many times someone is listed as a victim in a crime.

In [46]:
victim_df = pd.DataFrame(sorted(P.in_degree(), key = lambda x: x[1], reverse=True), columns=['Person','In Degree'])
victim_df.head(10)

,Person,In Degree
0,JonesEzekial,10
1,TatumAnna,10
2,GodfreyTalia,8
3,CandyCarol,7
4,EbersolShirley,7
5,SamuelLyle,6
6,AbramsChad,6
7,LeeCrystal,6
8,BrowningCalder,6
9,GradyRufus,6


Interestingly, we see a little overlap between suspects and victims of crimes.

In [47]:
test = pd.merge(left=suspect_df, right=victim_df, left_on='Person', right_on='Person')

test.head(11)

,Person,Out Degree,In Degree
0,WillisJenny,14,1
1,CarverJustin,10,0
2,HemphillBud,8,1
3,CarverJason,7,0
4,BallMelvin,7,0
5,MunsonKaty,6,1
6,AbramsChad,6,6
7,TuckerPaul,6,0
8,DicksonCarter,6,3
9,BurnleyAaron,6,0


Now we will apply the island method to look at how the suspects relate to their victims. We will first limit to those who have more than a single victim.

In [48]:
n_p2 = net.Network(height = "800px", width = "100%", notebook = True,
               heading = 'People (suspects with > 1 victim)', directed = True)

for n, v in P.nodes(data = True):
    if P.out_degree(n) > 1:
        n_p2.add_node(n, color = colors[v["Sex"]])
        for u,v,d in P.edges(n, data=True):
            n_p2.add_node(v)
            n_p2.add_edge(u,v,value=d["weight"])

n_p2.show("graph.html")

Now we'll look at the suspects who had the same victim more than once:

In [49]:
n_p3 = net.Network(height = "800px", width = "100%", notebook = True,
               heading = 'People (suspects with repeat victim)', directed = True)

for u,v,d in P.edges(data=True):
    if d["weight"] > 1:
        n_p3.add_node(u, color = colors[P.nodes[u]["Sex"]])
        n_p3.add_node(v, color = colors[P.nodes[v]["Sex"]])
        n_p3.add_edge(u,v,value=d["weight"])

n_p3.show("graph.html")

The above visual shows that there are but a few suspects who victimize the same person multiple times.

Next we will look at the relations where a suspect has been a victim of their victim. This might indicate some grudge or "beef" with one another.

In [50]:
n_p4 = net.Network(height = "800px", width = "100%", notebook = True,
               heading = 'Retaliation', directed = True)

for u,v,d in P.edges(data=True):
    if (v,u) in P.edges():
        n_p4.add_node(u, color = colors[P.nodes[u]["Sex"]])
        n_p4.add_node(v, color = colors[P.nodes[v]["Sex"]])
        n_p4.add_edge(u,v,value=d["weight"])

n_p4.show("graph.html")

In [52]:
# Compute centrality measures
pcc = pd.DataFrame(nx.closeness_centrality(P).items(), columns=['Person','Closeness Centrality'])
pbc = pd.DataFrame(nx.betweenness_centrality(P).items(), columns=['Person','Betweenness Centrality'])
pdci = pd.DataFrame(nx.in_degree_centrality(P).items(), columns=['Person','In Degree Centrality'])
pdco = pd.DataFrame(nx.out_degree_centrality(P).items(), columns=['Person','Out Degree Centrality'])

# Display all measures
data_frames = [suspect_df, victim_df, pcc, pbc, pdci, pdco]
pmeasures = reduce(lambda left,right: pd.merge(left,right,on=['Person']), data_frames)

pmeasures['Rank'] = pmeasures['Out Degree Centrality'].rank(ascending=False)
pmeasures.sort_values(by=['Rank']).head(10)

,Person,Out Degree,In Degree,Closeness Centrality,Betweenness Centrality,In Degree Centrality,Out Degree Centrality,Rank
0,WillisJenny,14,1,0.001412,0.000040,0.001412,0.019774,1.0
1,CarverJustin,10,0,0.000000,0.000000,0.000000,0.014124,2.0
2,HemphillBud,8,1,0.001412,0.000040,0.001412,0.011299,3.0
3,CarverJason,7,0,0.000000,0.000000,0.000000,0.009887,4.5
4,BallMelvin,7,0,0.000000,0.000000,0.000000,0.009887,4.5
10,AndersonWinston,6,0,0.000000,0.000000,0.000000,0.008475,8.5
8,DicksonCarter,6,3,0.004237,0.000040,0.004237,0.008475,8.5
9,BurnleyAaron,6,0,0.000000,0.000000,0.000000,0.008475,8.5
6,AbramsChad,6,6,0.013242,0.000180,0.008475,0.008475,8.5
5,MunsonKaty,6,1,0.001412,0.000014,0.001412,0.008475,8.5


### Crime Analysis


Unlike the projected graph for people, the crime projected graph is less interpretable. That is because of the directed nature of the bipartite graph. For example, two crimes will only be related when the victim of crime "A" is a suspect in crime "B".

First we will look at the crimes with the highest 

In [53]:
crime_df = pd.DataFrame(sorted(C.out_degree(), key = lambda x: x[1], reverse=True), columns=['Crime','Out Degree'])
crime_df.head(10)

,Crime,Out Degree
0,C23,23
1,C550,23
2,C525,23
3,C119,19
4,C5,17
5,C19,17
6,C7,17
7,C431,15
8,C426,15
9,C279,14


Let's look deeper at the top result, Crime "C550". A degree of 23 is rather large.

In [54]:
crime[crime["Crime"]=="C550"]

,Person,Crime,Name,Sex,Role
1452,815,C550,WillisJenny,F,Victim


Interestingly, there is only one person associated, "WillisJenny". However, when we look her name up we see that there are many crimes where she is listed as a suspect:

In [55]:
crime[crime["Name"]=="WillisJenny"]

,Person,Crime,Name,Sex,Role
1434,815,C511,WillisJenny,F,Suspect
1435,815,C513,WillisJenny,F,Suspect
1436,815,C529,WillisJenny,F,Suspect
1437,815,C546,WillisJenny,F,Suspect
1438,815,C308,WillisJenny,F,Suspect
1439,815,C489,WillisJenny,F,Suspect
1440,815,C473,WillisJenny,F,Suspect
1441,815,C525,WillisJenny,F,Victim
1442,815,C400,WillisJenny,F,Suspect
1443,815,C133,WillisJenny,F,Suspect


Now we'll look at crime in degrees. In this context that connection represents how many victims of other crimes became suspects.

In [56]:
crime_df = pd.DataFrame(sorted(C.in_degree(), key = lambda x: x[1], reverse=True), columns=['Crime','In Degree'])
crime_df.head(10)

,Crime,In Degree
0,C189,15
1,C34,10
2,C109,10
3,C169,9
4,C101,9
5,C106,9
6,C99,8
7,C358,7
8,C469,7
9,C470,7


Let's look at the top example:

In [57]:
crime[crime["Crime"]=="C189"]

,Person,Crime,Name,Sex,Role
258,124,C189,CandyCarol,F,Victim
275,131,C189,CarletonBonnie,F,Suspect
307,152,C189,ChambersGina,F,Suspect
730,404,C189,JeffersonArnold,M,Suspect
743,406,C189,JeffersonMillie,F,Suspect
860,472,C189,LittleJessie,F,Suspect
1096,610,C189,PotterAntonio,M,Suspect
1408,798,C189,WhartonKelvin,M,Suspect


Any of those suspects who were a victim in another crime creates the edge. For example:

In [58]:
crime[crime["Name"] == "LittleJessie"]

,Person,Crime,Name,Sex,Role
860,472,C189,LittleJessie,F,Suspect
861,472,C444,LittleJessie,F,Victim
862,472,C445,LittleJessie,F,Victim


In [59]:
# Compute centrality measures
ccc = pd.DataFrame(nx.closeness_centrality(C).items(), columns=['Crime','Closeness Centrality'])
cbc = pd.DataFrame(nx.betweenness_centrality(C).items(), columns=['Crime','Betweenness Centrality'])
cdci = pd.DataFrame(nx.in_degree_centrality(C).items(), columns=['Crime','In Degree Centrality'])
cdco = pd.DataFrame(nx.out_degree_centrality(C).items(), columns=['Crime','Out Degree Centrality'])

# Display all measures
data_frames = [crime_df, ccc, cbc, cdci, cdco]
pmeasures = reduce(lambda left,right: pd.merge(left,right,on=['Crime']), data_frames)

pmeasures['Rank'] = pmeasures['In Degree Centrality'].rank(ascending=False)
pmeasures.sort_values(by=['Rank']).head(10)

,Crime,In Degree,Closeness Centrality,Betweenness Centrality,In Degree Centrality,Out Degree Centrality,Rank
0,C189,15,0.027430,0.000000,0.027322,0.000000,1.0
1,C34,10,0.018735,0.000179,0.018215,0.014572,2.5
2,C109,10,0.018790,0.000000,0.018215,0.000000,2.5
3,C169,9,0.016393,0.000000,0.016393,0.000000,5.0
4,C101,9,0.017102,0.000086,0.016393,0.003643,5.0
5,C106,9,0.017102,0.000000,0.016393,0.000000,5.0
6,C99,8,0.015429,0.000000,0.014572,0.007286,7.0
10,C320,7,0.012953,0.000000,0.012750,0.000000,9.5
9,C470,7,0.012953,0.000000,0.012750,0.000000,9.5
7,C358,7,0.013413,0.000000,0.012750,0.000000,9.5
